In [1]:
import os, sys
current_directory = os.getcwd()
cwd_path = os.path.abspath(os.path.join(current_directory, '..'))

os.chdir(cwd_path)

In [ ]:
from src.browser.facebook_automation_workflow import FacebookAutomationWorkflow

##
##  self <=> workflow
##
workflow = FacebookAutomationWorkflow()

2025-09-27 14:37:55,959 - INFO - driver.py:109 in setup_driver - Browser driver initialized successfully
2025-09-27 14:37:55,960 - INFO - facebook_automation_workflow.py:142 in __init__ - Is self.browser a NoneType? False


In [3]:
if not workflow._login():
    raise RuntimeError("Failed to login to Facebook")

2025-09-27 14:43:33,713 - INFO - facebook_automation_workflow.py:225 in _login - Attempting Facebook login...
2025-09-27 14:43:33,714 - INFO - facebook.py:42 in login - Login attempt 1/3
2025-09-27 14:44:04,080 - INFO - facebook.py:149 in _is_login_successful - Navigation bar found; login successful
2025-09-27 14:44:05,115 - INFO - driver.py:302 in save_cookies - Cookies saved to e:\programming\python\facebooksummarizer\database\cookies\correodesconocidonumero293_at_gmail_dot_com.pkl
2025-09-27 14:44:05,115 - INFO - facebook.py:105 in login - Login successful; cookies saved
2025-09-27 14:44:05,116 - INFO - facebook_automation_workflow.py:228 in _login - Login successful!


In [4]:
from src.config.config import Config

##
##  From: _process_page
##

page_name = "redgol"
print(f"Processing page: {page_name}")

# Navigate to page
page_url = f"{Config.SOURCE_URL}{page_name}"
workflow.browser.get(page_url)
workflow._wait_random(
    workflow.config.min_delay_seconds,
    workflow.config.max_delay_seconds
)
workflow.logger.info(f"Navigated to {page_url}")

# Load posts
posts = workflow._load_initial_posts()
if not posts:
    workflow.logger.warning(f"No posts found on page {page_name}")

Processing page: redgol


2025-09-27 14:54:08,790 - INFO - 1383639967.py:17 in <module> - Navigated to https://www.facebook.com/redgol
2025-09-27 14:54:13,191 - INFO - facebook.py:180 in select_posts - Selected 4 posts
2025-09-27 14:54:13,192 - INFO - facebook_automation_workflow.py:333 in _load_initial_posts - Found 4 posts on attempt 1


In [5]:
from  src.browser.facebook_automation_workflow import PostData

##
## From: _process_single_post
##

post_element = posts[0]
post_data = PostData(page_name=page_name)

# Extract post ID
post_data.post_id = workflow._extract_post_id(post_element)
if not post_data.post_id:
    print("Failed to extract post ID")
else:
    print(f"Extracted post ID: {post_data.post_id}")

2025-09-27 14:54:27,580 - INFO - facebook_automation_workflow.py:445 in _extract_post_id - Extracted post ID: pfbid0qD6C8KCJV1zZRzMTo6MpwJvwgu5Pr51H4qeZYbEro2omW8goNAT614yReG3pCCfsl


Extracted post ID: pfbid0qD6C8KCJV1zZRzMTo6MpwJvwgu5Pr51H4qeZYbEro2omW8goNAT614yReG3pCCfsl


In [6]:
# Open post in new tab
if not workflow._open_post_in_new_tab(page_name, post_data.post_id):
    print("Failed to open post in new tab")

2025-09-27 14:54:32,662 - INFO - facebook_automation_workflow.py:465 in _open_post_in_new_tab - Opening post pfbid0qD6C8KCJV1zZRzMTo6MpwJvwgu5Pr51H4qeZYbEro2omW8goNAT614yReG3pCCfsl in new tab...
2025-09-27 14:54:32,663 - INFO - facebook.py:414 in open_post_in_new_tab - Opening post URL in new tab: https://www.facebook.com/redgol/posts/pfbid0qD6C8KCJV1zZRzMTo6MpwJvwgu5Pr51H4qeZYbEro2omW8goNAT614yReG3pCCfsl
2025-09-27 14:54:38,668 - INFO - facebook_automation_workflow.py:468 in _open_post_in_new_tab - Opened post URL: https://www.facebook.com/redgol/posts/pfbid0qD6C8KCJV1zZRzMTo6MpwJvwgu5Pr51H4qeZYbEro2omW8goNAT614yReG3pCCfsl


In [9]:
from selenium.webdriver.common.by import By

##
## From: _extract_post_content
##

post_box = workflow.browser.find_element(
    By.CSS_SELECTOR,
    'div[aria-labelledby][role="dialog"]'
)

# Extract post text
post_data.post_text = workflow._extract_post_text(post_box)

# Extract article URL using multiple strategies
post_data.article_url = workflow._extract_article_url(post_box, post_data.post_text)

2025-09-27 14:59:24,153 - INFO - facebook_automation_workflow.py:542 in _extract_post_text - Extracted post text: ¡PAR-TI-DA-ZO! El VIDEO y DETALLES del triunfazo del Atlético ante Real Madrid en el primer comentar...
2025-09-27 14:59:24,154 - INFO - facebook_automation_workflow.py:572 in _extract_article_url - Trying post_card extraction...
2025-09-27 14:59:34,161 - INFO - facebook_automation_workflow.py:572 in _extract_article_url - Trying post_text extraction...
2025-09-27 14:59:34,161 - INFO - facebook_automation_workflow.py:572 in _extract_article_url - Trying comment_section extraction...
2025-09-27 14:59:34,209 - INFO - facebook_automation_workflow.py:575 in _extract_article_url - Found URL via comment_section: https://l.facebook.com/l.php?u=https%3A%2F%2Fredgol.cl%2Finternacional%2Fatletico-vs-real-madrid-resumen-resultado-y-goles-de-julian-alvarez-en-la-liga%3Ffbclid%3DIwZXh0bgNhZW0CMTAAYnJpZBEwMTFDSWlDbDBwQTE2UTNhcwEecGNQHUiwyGfu_3ScJLEIzOsqiV_rjGMSMIUttaVY-CiBpGCaiZdsvuyy3tw

In [10]:
# Process article if URL found
if post_data.article_url:
    post_data.article_text = workflow._scrape_article(post_data.article_url)
    
    if post_data.article_text and post_data.post_text:
        post_data.analysis = workflow._analyze_content(
            post_data.post_text,
            post_data.article_text
        )
        
        # Post comment if enabled and analysis successful
        if post_data.analysis and post_data.analysis.get('output'):
            workflow._post_comment(post_data.analysis['output'])

2025-09-27 14:59:58,410 - INFO - facebook_automation_workflow.py:655 in _scrape_article - Scraping article: https://l.facebook.com/l.php?u=https%3A%2F%2Fredgol.cl%2Finternacional%2Fatletico-vs-real-madrid-resumen-resultado-y-goles-de-julian-alvarez-en-la-liga%3Ffbclid%3DIwZXh0bgNhZW0CMTAAYnJpZBEwMTFDSWlDbDBwQTE2UTNhcwEecGNQHUiwyGfu_3ScJLEIzOsqiV_rjGMSMIUttaVY-CiBpGCaiZdsvuyy3tw_aem_2CdUFp94ROrwEkzvPGQIOg&h=AT09Y2GlvVtTvJI_OC3E_6PjT026_Lv-012Xczl2t47BMD440RLRlnD02SVMOvb7qtkzqAilqBrXPbJjk8mLX5_kIyCBwvPWQLTRL8ARvKOAJP0w4hm3a5gcfJt0AzqasQ5owYYOP8kc-KtA&__tn__=R]-R&c[0]=AT1bZ1dMjbFHlGOboRHfqxxnCUYyFV601Xx8kYNZWBZPXEbuOxsLs9Rrcp-jf09ZzJnb-AiKRTCNvAwIs8NvFpiaEqiCZug0o4SphaLB0vERU2tGvi036Wyt9E5rj6fWZqjOFb_SlvFqA5T-BGxSzGJufGVP13BicXosiJVKcSHRK2tvrwhkfHSIBV32AAM
2025-09-27 15:00:05,369 - INFO - facebook_automation_workflow.py:667 in _scrape_article - Scraped article text: 2755 characters
2025-09-27 15:00:05,370 - INFO - facebook_automation_workflow.py:695 in _analyze_content - Analyzing content

In [11]:
# Close post tab
workflow.browser.close_current_tab()

In [12]:
workflow.close()
workflow = None

2025-09-27 15:01:19,644 - INFO - driver.py:428 in close - Browser driver closed
